### Pegando as músicas de cada artista da MPB

In [5]:
import os
import pandas as pd
import platform
import selenium
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
dicionario_musicas = {'indice_musica': [],
                     'titulo_musica': [],
                     'nome_artista': [],
                     'artistas_participacao': [],
                     'letra': [],
                     'album': [],
                     'data_lancamento': [],
                     'tags': [],
                     'identificador' : []}

musicas_df = pd.DataFrame.from_dict(dicionario_musicas)
nome_arquivo_musicas = 'musicas.csv'
indice_musica = 0
if os.path.isfile(nome_arquivo_musicas):
    musicas_df = pd.read_csv(nome_arquivo_musicas)
    if len(musicas_df) > 0:
        indice_musica = list(musicas_df['indice_musica'])[-1]
else:
    musicas_df.to_csv(nome_arquivo_musicas, index=False)

print(indice_musica)

999


In [9]:
sistema_operacional = platform.system()

if sistema_operacional == 'Linux':
    caminho_webdriver = os.path.join('chromedriver_linux64', 'chromedriver')
elif sistema_operacional == 'Windows':
    caminho_webdriver = os.path.join('chromedriver_win32', 'chromedriver.exe')

servico = Service(executable_path=caminho_webdriver)

In [11]:
opcoes = webdriver.ChromeOptions()
navegador = webdriver.Chrome(service=servico, options = opcoes)
espera_navegador = WebDriverWait(navegador, 20)
identificadores = list(musicas_df['identificador'])

pagina_musicas_mpb = 'https://genius.com/tags/mpb/all?page=1'
navegador.get(pagina_musicas_mpb)

while True:
    navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    if not (len(navegador.find_elements(By.XPATH, "//a[contains(text(), 'more')]"))) > 0:
        break

elementos_musica_id = '//span[@class="title_with_artists"]'
elementos_musica = espera_navegador.until(EC.presence_of_all_elements_located((By.XPATH, elementos_musica_id)))

for j in range(indice_musica, len(elementos_musica)):
    musica = elementos_musica[j]
    elemento_pai = musica.find_element(By.XPATH, '..')
    titulo_musica = musica.find_element(By.CLASS_NAME, 'song_title').get_attribute("innerText")

    elemento_artistas = musica.find_element(By.CLASS_NAME, 'artist_name')
    nome_artista = elemento_artistas.find_element(By.CLASS_NAME, 'primary_artist_name').get_attribute("innerText")

    identificador = f'{titulo_musica} {nome_artista}'

    if identificador not in identificadores: 
        lista_artistas_destacados = []
        try:
            nomes_artistas_destacados = elemento_artistas.find_elements(By.CLASS_NAME, 'featured_artist_name')
            for nome_artista_destacado in nomes_artistas_destacados:
                lista_artistas_destacados.append(nome_artista_destacado.get_attribute("innerText"))
        except Exception as e:
            print(f'Erro de artistas destacados: {str(e)}')
            
        link_letra = elemento_pai.get_attribute('href')
        navegador.execute_script(f'window.open("{link_letra}","_blank");')
        navegador.switch_to.window(navegador.window_handles[1])

        letra = ''
        try:
            identificador_containers_letra_id = '//div[@data-lyrics-container="true"]'
            containers_letra = espera_navegador.until(EC.presence_of_all_elements_located((By.XPATH, identificador_containers_letra_id)))
            for container in containers_letra:
                letra_atual = container.get_attribute("innerText")
                letra += letra_atual
        except Exception as e:
            print(f'Erro na letra: {str(e)}')

        album = ''
        try:
            album = navegador.find_element(By.XPATH, "//a[contains(@class, 'PrimaryAlbum')]").get_attribute("innerText")
        except Exception as e:
            print(f'Erro no nome do álbum: {str(e)}')

        data_lancamento = ''
        try:
            identificador_data_lancamento = "//span[contains(@class, 'MetadataStats')]"
            elementos_data_lancamento = navegador.find_elements(By.XPATH, identificador_data_lancamento)
            data_lancamento = elementos_data_lancamento[0].get_attribute("innerText")
        except Exception as e:
            print(f'Erro na data de lançamento: {str(e)}')

        tags = ''
        try:
            identificador_tags = "//a[contains(@class, 'SongTags__Tag')]"
            elementos_tags = navegador.find_elements(By.XPATH, identificador_tags)
            tags = ','.join([tag.get_attribute("innerText") for tag in elementos_tags])
        except Exception as e:
            print(f'Erro nas tags: {str(e)}')

        nova_linha = [j, titulo_musica, nome_artista, lista_artistas_destacados, letra, album, data_lancamento, tags, identificador]
        df_nova_linha = pd.DataFrame([nova_linha])
        df_nova_linha.to_csv(nome_arquivo_musicas, mode='a', header=False, index=False)
        identificadores.append(identificador)
        navegador.close()
        navegador.switch_to.window(navegador.window_handles[0])

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro na letra: Message: 

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpat

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'PrimaryAlbum')]"}
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x006AA813+48355]
	(No symbol) [0x0063C4B1]
	(No symbol) [0x00545358]
	(No symbol) [0x005709A5]
	(No symbol) [0x00570B3B]
	(No symbol) [0x0059E232]
	(No symbol) [0x0058A784]
	(No symbol) [0x0059C922]
	(No symbol) [0x0058A536]
	(No symbol) [0x005682DC]
	(No symbol) [0x005693DD]
	GetHandleVerifier [0x0090AABD+2539405]
	GetHandleVerifier [0x0094A78F+2800735]
	GetHandleVerifier [0x0094456C+2775612]
	GetHandleVerifier [0x007351E0+616112]
	(No symbol) [0x00645F8C]
	(No symbol) [0x00642328]
	(No symbol) [0x0064240B]
	(No symbol) [0x00634FF7]
	BaseThreadInitThunk [0x76FB7D59+25]
	RtlInitializeExceptionChain [0x77DDB79B+107]
	RtlClearBits [0x77DDB71F+191]

Erro no nome do álbum: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contain

In [12]:
mdf = pd.read_csv('musicas.csv')

In [13]:
mdf

,indice_musica,titulo_musica,nome_artista,artistas_participacao,letra,album,data_lancamento,tags,identificador
0,0,Trevo (Tu),ANAVITÓRIA,['Tiago\xa0Iorc'],"[Letra de ""Trevo (Tu)"" por Anavitória e Tiago ...",ANAVITÓRIA (2016),"Aug. 18, 2016","Pop,MPB (Música Popular Brasileira),Latin Musi...",Trevo (Tu) ANAVITÓRIA
1,1,Tudo que você podia ser,Milton Nascimento,[],[Verso 1]\nCom sol e chuva\nVocê sonhava\nQue ...,Perfil (2006),"Jan. 1, 1972","Pop,Folk Pop,Sunshine Pop,MPB (Música Popular ...",Tudo que você podia ser Milton Nascimento
2,2,Cálice,Chico Buarque,['Milton\xa0Nascimento'],"[Letra de ""Cálice"" com Chico Buarque & Milton ...",Chico Buarque (1978) (1978),"Jan. 1, 1978","Rock,MPB (Música Popular Brasileira),Em Portug...",Cálice Chico Buarque
3,3,Agora Eu Quero Ir,ANAVITÓRIA,[],"Letra de ""Agora Eu Quero Ir"" por Anavitória\n\...",ANAVITÓRIA (2016),"Aug. 18, 2016","Pop,MPB (Música Popular Brasileira),Em Portugu...",Agora Eu Quero Ir ANAVITÓRIA
4,4,Porque Eu Te Amo,ANAVITÓRIA,[],"[Letra de ""Porque Eu Te Amo""]\n\n[Verso 1: Ana...",O Tempo É Agora (2018),"Aug. 3, 2018","Pop,MPB (Música Popular Brasileira),Brasil,Em ...",Porque Eu Te Amo ANAVITÓRIA
...,...,...,...,...,...,...,...,...,...
995,995,Loucos de Cara,Vitor Ramil,[],"[Gancho]\nVem, anda comigo pelo planeta\nVamos...",Tango (1987),1987,"Pop,MPB (Música Popular Brasileira),Brasil,Em ...",Loucos de Cara Vitor Ramil
996,996,Rugas,Nelson Cavaquinho,[],"[Letra de ""Rugas"" com Nelson Cavaquinho]\n\nSe...",Depoimento do Poeta (1970),1970,"Pop,MPB (Música Popular Brasileira),Brasil,Sam...",Rugas Nelson Cavaquinho
997,997,Nanã,Wado,['OTTO'],NaN,"A Beleza que Deriva do Mundo, Mas a Ele Escapa...","Oct. 2, 2020","Rock,MPB (Música Popular Brasileira),Em Portug...",Nanã Wado
998,998,Me Deixas Louca,Illy (BRA),['Baco\xa0Exu do Blues'],"[Letra de ""Me Deixas Louca"" com Illy & Baco Ex...",Te Adorando pelo Avesso (2020),"Apr. 23, 2020","Pop,Brasil,MPB (Música Popular Brasileira),Em ...",Me Deixas Louca Illy (BRA)
